### This notebook runs FlowSOM pixel-level clustering

In [1]:
# import required packages
import os

import sys
sys.path.append('..')
sys.path.append('../ark')

import numpy as np
import pandas as pd
import xarray as xr

from ark.phenotyping import preprocess
from ark.phenotyping import cluster
from ark.utils import io_utils, load_utils

### Set file paths and parameters

In [2]:
# assign file paths
base_dir = "/Users/alexkong/Downloads/granulomaCohort_allData"
tiff_dir = os.path.join(base_dir, "TIFs")
all_data_path = os.path.join(base_dir, "cohortDatav.csv")

In [3]:
# set mibitiff parameters
MIBItiff = False
mibitiff_suffix = '-MassCorrected-Filtered.tiff'

In [4]:
# either get all fovs in the folder...
if MIBItiff:
    fovs = io_utils.list_files(tiff_dir, substrs=MIBItiff_suffix)
else:
    fovs = io_utils.list_folders(tiff_dir)

# ... or optionally, select a specific set of fovs manually
# fovs = ["fov1", "fov2"]

In [5]:
# set the channels to subset over
chan_list = ["CD45", "SMA", "Vimentin", "CD31", "Keratin-pan", "E-cadherin", "MastChyTry",
             "MPO", "CD20", "CD3", "CD14", "HLA-DR-DQ-DP", "Foxp3", "CD16", "CD11c", "CD206",
             "CD11b", "CD68", "CD163", "CD209"]

### Load data

In [6]:
# load the image data
if MIBItiff:
    img_xr = load_utils.load_imgs_from_mibitiff(tiff_dir, mibitiff_files=fovs, channels=chan_list, dtype="int16")
else:
    img_xr = load_utils.load_imgs_from_tree(tiff_dir, img_sub_folder="TIFsNoAgg", fovs=fovs, channels=chan_list, dtype="int16")

In [7]:
# load the data cohort
all_data = pd.read_csv(all_data_path)

In [8]:
# load the segmentation labels
segmentation_labels = load_utils.load_imgs_from_dir(base_dir,
                                                    xr_dim_name='compartments',
                                                    xr_channel_names=['whole_cell'],
                                                    force_ints=True)

In [9]:
# to make data types consistent, remove 'Point' from segmentation label fov coordinate names
segmentation_labels.coords['fovs'].values = [fov.replace('segmentationmask_SampleID', 'Point') for fov in segmentation_labels.coords['fovs'].values]

### Preprocess

In [10]:
# run FlowSOM preprocessing
flowsom_data = preprocess.create_pixel_matrix(img_xr, segmentation_labels)

In [13]:
flowsom_data[flowsom_data['fov'] == 'Point6'].head(10)

,CD45,SMA,Vimentin,CD31,Keratin-pan,E-cadherin,MastChyTry,MPO,CD20,CD3,...,CD11c,CD206,CD11b,CD68,CD163,CD209,fov,row_index,column_index,segmentation_label
0,0.015399,0.0,0.658230,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.0,0.158448,0.0,0.0,Point6,0,0,1
1,0.021428,0.0,0.635231,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.0,0.159286,0.0,0.0,Point6,0,1,1
2,0.032239,0.0,0.594589,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.0,0.163196,0.0,0.0,Point6,0,2,1
3,0.045881,0.0,0.546373,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.0,0.171642,0.0,0.0,Point6,0,3,1
4,0.060021,0.0,0.503680,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000e+00,0.0,0.0,0.182778,0.0,0.0,Point6,0,4,1
5,0.072726,0.0,0.476962,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,8.229230e-08,0.0,0.0,0.192394,0.0,0.0,Point6,0,5,1
6,0.083820,0.0,0.467648,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,...,1.496675e-06,0.0,0.0,0.197381,0.0,0.0,Point6,0,6,1
7,0.094931,0.0,0.469190,0.0,0.0,0.0,0.0,0.0,0.0,0.000038,...,1.131804e-05,0.0,0.0,0.196612,0.0,0.0,Point6,0,7,1
8,0.107756,0.0,0.473696,0.0,0.0,0.0,0.0,0.0,0.0,0.000249,...,6.480598e-05,0.0,0.0,0.188491,0.0,0.0,Point6,0,8,1
9,0.123040,0.0,0.476382,0.0,0.0,0.0,0.0,0.0,0.0,0.001210,...,2.956077e-04,0.0,0.0,0.170875,0.0,0.0,Point6,0,9,1


In [15]:
# write data to csv, start at "Cluster the data" after this
flowsom_data.to_csv(os.path.join('pixel_results', 'example_pixel_matrix.csv'), index=False)

### Train the SOM

In [2]:
# use preprocessed data
flowsom_data = pd.read_csv(os.path.join('pixel_results', 'example_pixel_matrix.csv'))

In [ ]:
# train the som, uses parameters from R SOM package (10 x_neurons, 10 y_neurons, 10 passes through the dataset)
weights = cluster.train_som(flowsom_data[chan_list], x_neurons=10, y_neurons=10, num_passes=10)

In [9]:
np.save(os.path.join('pixel_results', 'som_weights.npy'), weights)

### Cluster the data

In [8]:
weights = np.load(os.path.join('pixel_results', 'som_weights.npy'))

In [10]:
# cluster the data
flowsom_data['cluster_label'] = cluster.cluster_som(flowsom_data[chan_list], weights)

In [11]:
flowsom_data['cluster_label'].value_counts()

1     633959
3     373880
4     329334
38    286675
14    219845
39    186599
7     161918
20    146521
0     123981
22    100177
35     82594
5      74916
21     69925
2      64402
12     59678
6      58858
18     58834
13     58785
33     54447
23     54100
15     52579
44     45131
19     42588
11     40231
66     36927
16     36180
47     36147
51     32914
52     27765
9      27676
       ...  
87      3402
72      3214
74      3143
59      3008
82      2935
45      2935
29      2922
85      2838
75      2761
92      2757
61      2633
73      2483
50      2347
83      1958
77      1895
46      1659
99      1649
64      1448
86      1394
79      1393
71      1330
84      1277
78       788
97       704
96       677
80       641
94       597
91       493
98       322
88       245
Name: cluster_label, Length: 100, dtype: int64